In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa
import cv2
from ultralytics import YOLO
import tensorflow as tf
import gc
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score, precision_score, f1_score

In [4]:
def get_unet_mask(image_path, Unet):
    """
    Genera una máscara de segmentación utilizando un modelo Unet.

    Parámetros:
    image_path (str): Ruta de la imagen de entrada.
    Unet (modelo): Modelo Unet preentrenado.

    Retorna:
    numpy.ndarray: Máscara de segmentación binaria.
    """
    image = cv2.imread(image_path)
    img_orig = image.astype(np.float32)
    img_orig_gray = cv2.cvtColor(img_orig, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img_orig_gray, (256, 512))
    normalizedImg = np.zeros(img.shape)
    img = cv2.normalize(img, normalizedImg, -1, 1, cv2.NORM_MINMAX)
    img = img[None, ..., None]

    pred_maps, seg_pred = Unet.predict(img, verbose=0)
    mask = np.asarray(np.squeeze(seg_pred))
    mask = np.round(mask)

    mask = cv2.resize(mask, (img_orig_gray.shape[1], img_orig_gray.shape[0]))
    mask = mask.astype(bool)
    return mask

In [20]:
def get_yolo_mask(image_path, model):
    """
    Genera una máscara de segmentación utilizando un modelo YOLO.

    Parámetros:
    image_path (str): Ruta de la imagen de entrada.
    model (YOLO): Modelo YOLO preentrenado.

    Retorna:
    numpy.ndarray: Máscara de segmentación binaria.
    """
    try:
        image = cv2.imread(image_path)
        original_shape = image.shape[:2]
        results = model(image)

        mask = np.zeros(original_shape, dtype=np.uint8)

        if results[0].masks is not None:
            masks = results[0].masks.xy
            for mask_array in masks:
                if mask_array.shape[0] == 0:  # Manejar el caso de máscaras vacías
                    continue
                mask_array = mask_array.astype(np.int32)
                cv2.fillPoly(mask, [mask_array], 1)
        else:
            print("No masks found in the results")

        mask = mask.astype(bool)
        return mask
    except Exception as e:
        print(f"Error in get_yolo_mask for image {image_path}: {e}")
        return None

In [24]:
def calculate_iou(mask1, mask2):
    """
    Calcula el Índice de Superposición (IoU) entre dos máscaras binarias.

    Parámetros:
    mask1 (numpy.ndarray): Primera máscara binaria.
    mask2 (numpy.ndarray): Segunda máscara binaria.

    Retorna:
    float: Valor del IoU.
    """
    if np.sum(mask1) == 0 and np.sum(mask2) == 0:
        return 1.0  # Ambas máscaras son cero, IoU es 1
    elif np.sum(mask1) == 0 or np.sum(mask2) == 0:
        return 0.0  # Una de las máscaras es cero, IoU es 0
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    iou = np.sum(intersection) / np.sum(union) 
    return iou
def calculate_recall(ground_truth_mask, predicted_mask):
    """
    Calcula el Recall entre dos máscaras binarias.

    Parámetros:
    ground_truth_mask (numpy.ndarray): Máscara de referencia (ground truth).
    predicted_mask (numpy.ndarray): Máscara predicha.

    Retorna:
    float: Valor del Recall.
    """
    return recall_score(ground_truth_mask.flatten(), predicted_mask.flatten())

def calculate_precision(ground_truth_mask, predicted_mask):
    """
    Calcula la Precision entre dos máscaras binarias.

    Parámetros:
    ground_truth_mask (numpy.ndarray): Máscara de referencia (ground truth).
    predicted_mask (numpy.ndarray): Máscara predicha.

    Retorna:
    float: Valor del Precision.
    """
    return precision_score(ground_truth_mask.flatten(), predicted_mask.flatten())

def calculate_dice(ground_truth_mask, predicted_mask):
    """
    Calcula el coeficiente DICE entre dos máscaras binarias.

    Parámetros:
    ground_truth_mask (numpy.ndarray): Máscara de referencia (ground truth).
    predicted_mask (numpy.ndarray): Máscara predicha.

    Retorna:
    float: Valor del coeficiente DICE.
    """
    if np.sum(ground_truth_mask) == 0 and np.sum(predicted_mask) == 0:
        return 1.0  # Ambas máscaras son cero, IoU es 1
    
    intersection = np.logical_and(ground_truth_mask, predicted_mask)
    dice = 2 * np.sum(intersection) / (np.sum(ground_truth_mask) + np.sum(predicted_mask))
    return dice

def calculate_f1(ground_truth_mask, predicted_mask):
    """
    Calcula el F1-Score entre dos máscaras binarias.

    Parámetros:
    ground_truth_mask (numpy.ndarray): Máscara de referencia (ground truth).
    predicted_mask (numpy.ndarray): Máscara predicha.

    Retorna:
    float: Valor del F1-Score.
    """
    return f1_score(ground_truth_mask.flatten(), predicted_mask.flatten())

In [25]:
def evaluate_model_yolo(test_dir, yolo_model, num_images=3500):
    """
    Evalúa el modelo YOLO en un conjunto de datos de prueba y calcula el IoU promedio.

    Parámetros:
    test_dir (str): Directorio que contiene las imágenes de prueba y sus máscaras correspondientes.
    yolo_model (YOLO): Modelo YOLO preentrenado.
    num_images (int): Número de imágenes a procesar.

    Retorna:
    None
    """
    yolo_ious = []
    yolo_recalls = []
    yolo_precision = []
    yolo_dice = []
    yolo_f1 = []
    zero_iou_count = 0
    iou_bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    iou_histogram = np.zeros(len(iou_bins) - 1)
    for idx in range(num_images):
            try:
                image_name = f"{idx}.png"
                mask_name = f"{idx}_seg.png"
                mask_path = os.path.join(test_dir, mask_name)
                image_path = os.path.join(test_dir, image_name)

                ground_truth_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                ground_truth_mask = ground_truth_mask.astype(bool)


                yolo_mask = get_yolo_mask(image_path, yolo_model)
                if yolo_mask is None:
                    print(f"Warning: No YOLO mask generated for image {image_path}")
                    continue

                # Redimensionar la máscara de YOLO a las dimensiones de la máscara de referencia
                yolo_mask_resized = cv2.resize(yolo_mask.astype(np.uint8), (ground_truth_mask.shape[1], ground_truth_mask.shape[0])).astype(bool)


                iou_yolo = calculate_iou(ground_truth_mask, yolo_mask_resized)
                recall_yolo = calculate_recall(ground_truth_mask, yolo_mask_resized)
                precision_yolo = calculate_precision(ground_truth_mask, yolo_mask_resized)
                dice_yolo = calculate_dice(ground_truth_mask, yolo_mask_resized)
                f1_yolo = calculate_f1(ground_truth_mask, yolo_mask_resized)
                
                if np.isnan(iou_yolo):
                    print(f"Warning: IoU is NaN for image {image_path}")
                else:
                    yolo_ious.append(iou_yolo)
                    yolo_recalls.append(recall_yolo)
                    yolo_precision.append(precision_yolo)
                    yolo_dice.append(dice_yolo)
                    yolo_f1.append(f1_yolo)
                    if iou_yolo == 0.0:
                        zero_iou_count += 1
                    else:
                        for i in range(len(iou_bins) - 1):
                            if iou_bins[i] <= iou_yolo < iou_bins[i + 1]:
                                iou_histogram[i] += 1
                                break
                    

                # Liberar recursos
                del ground_truth_mask
                del yolo_mask
                del yolo_mask_resized
                gc.collect()
                tf.keras.backend.clear_session()
            except Exception as e:
                print(f"Error processing {image_name}: {e}")
                continue

    avg_iou_yolo = np.mean(yolo_ious) if yolo_ious else 0.0
    avg_recall = np.mean(yolo_recalls) if yolo_recalls else 0.0
    avg_precision = np.mean(yolo_precision) if yolo_precision else 0.0
    avg_dice = np.mean(yolo_dice) if yolo_dice else 0.0
    avg_f1 = np.mean(yolo_f1) if yolo_f1 else 0.0

    print(f"Average YOLO IoU: {avg_iou_yolo}")
    print(f"Average YOLO Recall: {avg_recall}")
    print(f"Average YOLO Precision: {avg_precision}")
    print(f"Average YOLO DICE: {avg_dice}")
    print(f"Average YOLO F1: {avg_f1}")
    print(f"IoU 0: {zero_iou_count}")
    plt.figure(figsize=(10, 5))
    plt.bar([f"{iou_bins[i]}-{iou_bins[i+1]}" for i in range(len(iou_bins) - 1)], iou_histogram, color='blue', edgecolor='black')
    plt.title('Distribution of IoUs')
    plt.xlabel('IoU Range')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# YOLO nano
test_dir = "/home/voicelab/Desktop/segmentation_glottis/BAGLS/test/test"
yolo_model_path = "/home/voicelab/Desktop/segmentation_glottis/YOLOV8/runs/segment/train2/weights/best.pt"
yolo_model = YOLO(yolo_model_path)
evaluate_model_yolo(test_dir, yolo_model)

In [ ]:
# YOLO Extra Large
test_dir = "/home/voicelab/Desktop/segmentation_glottis/BAGLS/test/test"
yolo_model_path = "/home/voicelab/Desktop/segmentation_glottis/YOLOV8/runs/segment/train/weights/best.pt"
yolo_model = YOLO(yolo_model_path)
evaluate_model_yolo(test_dir, yolo_model)

In [16]:

def evaluate_model_unet(test_dir, unet_model_path, num_images=3500):
    """
    Evalúa el modelo Unet en un conjunto de datos de prueba y calcula el IoU promedio.

    Parámetros:
    test_dir (str): Directorio que contiene las imágenes de prueba y sus máscaras correspondientes.
    unet_model_path (str): Ruta del modelo Unet preentrenado.
    batch_size (int): Número de imágenes a procesar en cada lote.

    Retorna:
    None
    """
    unet_model = load_model(unet_model_path, compile=False, custom_objects={'InstanceNormalization': tfa.layers.InstanceNormalization})
    
    unet_ious = []
    unet_recalls = []
    unet_precision = []
    unet_dice = []
    unet_f1 = []
    zero_iou_count = 0
    iou_bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    iou_histogram = np.zeros(len(iou_bins) - 1)
    for idx in range(num_images):
            try:
                image_name = f"{idx}.png"
                mask_name = f"{idx}_seg.png"
                mask_path = os.path.join(test_dir, mask_name)
                image_path = os.path.join(test_dir, image_name)

                ground_truth_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                ground_truth_mask = ground_truth_mask.astype(bool)
                unet_mask = get_unet_mask(image_path, unet_model) 
                if unet_mask  is None:
                    print(f"Warning: No YOLO mask generated for image {image_path}")
                    continue

                
                unet_mask_resized = cv2.resize(unet_mask.astype(np.uint8), (ground_truth_mask.shape[1], ground_truth_mask.shape[0])).astype(bool)


                iou_unet = calculate_iou(ground_truth_mask, unet_mask_resized)
                recall_unet = calculate_recall(ground_truth_mask, unet_mask_resized)
                precision_unet = calculate_precision(ground_truth_mask, unet_mask_resized)
                dice_unet = calculate_dice(ground_truth_mask, unet_mask_resized)
                f1_unet = calculate_f1(ground_truth_mask, unet_mask_resized)
                if np.isnan(iou_unet):
                    print(f"Warning: IoU is NaN for image {image_path}")
                else:
                    unet_ious.append(iou_unet)
                    unet_recalls.append(recall_unet)
                    unet_precision.append(precision_unet)
                    unet_dice.append(dice_unet)
                    unet_f1.append(f1_unet)
                    if iou_unet == 0.0:
                        zero_iou_count += 1
                    else:
                        for i in range(len(iou_bins) - 1):
                            if iou_bins[i] <= iou_unet < iou_bins[i + 1]:
                                iou_histogram[i] += 1
                                break


                del ground_truth_mask
                del unet_mask
                del unet_mask_resized

                gc.collect()
            except Exception as e:
                print(f"Error processing {image_name}: {e}")
                continue

    avg_iou_unet = np.mean(unet_ious)
    avg_recall = np.mean(unet_recalls) if unet_recalls else 0.0
    avg_precision = np.mean(unet_precision) if unet_precision else 0.0
    avg_dice = np.mean(unet_dice) if unet_dice else 0.0
    avg_f1 = np.mean(unet_f1) if unet_f1 else 0.0

    print(f"Average YOLO IoU: {avg_iou_unet}")
    print(f"Average YOLO Recall: {avg_recall}")
    print(f"Average YOLO Precision: {avg_precision}")
    print(f"Average YOLO DICE: {avg_dice}")
    print(f"Average YOLO F1: {avg_f1}")
    print(f"Average Unet IoU: {avg_iou_unet}")
    print(f"IoU 0: {zero_iou_count}")


In [ ]:

test_dir = "/home/voicelab/Desktop/segmentation_glottis/BAGLS/test/test"
unet_model_path = "/home/voicelab/Downloads/epoch025.h5"

evaluate_model_unet(test_dir, unet_model_path)